## Ciclo 02 - Aprendizado Supervisionado - Classificação

### 1.0 Import libraries

In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt

### 1.1 - Load Dataset

In [3]:
dataset_path = '../dataset/train.csv'

df_root = pd.read_csv(dataset_path)

df = df_root.copy()

In [4]:
df_root.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.172008,2577.05,24196.89636,104.306544,31.038763,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.874152,2465.39,19227.37796,69.858778,36.917093,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.643127,1055.29,42822.28223,134.201478,34.561714,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.141267,703.05,51786.82600,297.350067,31.493561,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.716114,891.29,44626.85346,134.201478,28.028887,2,8,7,24,10,8,20,Sim,Não,Negar


In [5]:
df.loc[:, 'limite_adicional'].unique()

array(['Negar', 'Conceder'], dtype=object)

In [6]:
# Porcentagem de Negar e Conceder da nossa medida de interessa "limite_adicional"
df.loc[:, 'limite_adicional'].value_counts(normalize = True)

limite_adicional
Negar       0.841579
Conceder    0.158421
Name: proportion, dtype: float64

### 2.0 Seleção de Features

In [7]:
df.columns

Index(['id_cliente', 'idade', 'saldo_atual', 'divida_atual', 'renda_anual',
       'valor_em_investimentos', 'taxa_utilizacao_credito', 'num_emprestimos',
       'num_contas_bancarias', 'num_cartoes_credito', 'dias_atraso_dt_venc',
       'num_pgtos_atrasados', 'num_consultas_credito', 'taxa_juros',
       'investe_exterior', 'pessoa_polit_exp', 'limite_adicional'],
      dtype='object')

In [8]:
# Seleção de Features
# x = features e y = label

label = 'limite_adicional'
features = ['idade', 'saldo_atual', 'divida_atual', 'renda_anual',
       'valor_em_investimentos', 'taxa_utilizacao_credito', 'num_emprestimos',
       'num_contas_bancarias', 'num_cartoes_credito', 'dias_atraso_dt_venc',
       'num_pgtos_atrasados', 'num_consultas_credito', 'taxa_juros']


x_train = df.loc[:, features] # Informações que eu irei me basear...
y_train = df.loc[:, label].values.ravel() # Para tentar chegar nessa informação.

### 3.0 Treinamento

In [12]:
# Definição de parâmetros do treinamento.
k = 7
knn_classifier = KNeighborsClassifier( n_neighbors = k )

# Treinamento do algoritmo
knn_classifier.fit(x_train, y_train)

# Previsão das observações
y_pred = knn_classifier.predict(x_train)

In [13]:
y_pred.shape

(9500,)

In [11]:
# Comparação de Previsto com o Realizado
df_result = df.loc[:, ['id_cliente', 'idade', 'limite_adicional']]
df_result['predicted'] = y_pred

In [10]:
df_result

,id_cliente,idade,limite_adicional,predicted
0,1767,21,Negar,Negar
1,11920,40,Negar,Negar
2,8910,36,Negar,Negar
3,4964,58,Negar,Negar
4,10100,35,Negar,Negar
...,...,...,...,...
9495,5155,29,Negar,Negar
9496,11977,1237,Negar,Negar
9497,9278,47,Negar,Negar
9498,2525,42,Negar,Negar


In [11]:
# Pegar resultados e tentar achar algum errado com sample
df_result.loc[:, :].sample(10)

,id_cliente,idade,limite_adicional,predicted
7360,4067,27,Conceder,Negar
6170,11546,46,Conceder,Negar
7077,9227,58,Negar,Conceder
4586,6500,42,Conceder,Negar
4270,5842,60,Negar,Negar
6172,1491,47,Conceder,Negar
27,2088,38,Negar,Negar
7180,9076,50,Conceder,Negar
3032,450,41,Negar,Negar
8697,2231,27,Negar,Negar


In [12]:
df_result.loc[:, 'predicted'].value_counts(normalize = True)

predicted
Negar       0.939789
Conceder    0.060211
Name: proportion, dtype: float64

In [23]:
# Aplicando função para criar uma nova coluna com a comparação entre o real e a previsão
df_result['acertos'] = (df_result.loc[:, ['id_cliente', 'limite_adicional', 'predicted']].apply(lambda x : 1 if x['limite_adicional'] == x['predicted'] else 0, axis = 1))

In [24]:
df_result.loc[:, :].sample(10)

,id_cliente,idade,limite_adicional,predicted,acertos
1497,7524,54,Negar,Negar,1
8051,4879,50,Conceder,Negar,0
1825,2110,22,Negar,Negar,1
6141,3993,22,Negar,Negar,1
1013,2772,21,Negar,Negar,1
29,4338,44,Negar,Negar,1
1468,6299,52,Negar,Negar,1
9496,11977,1237,Negar,Negar,1
4511,12226,30,Negar,Negar,1
2892,6837,31,Negar,Negar,1


In [25]:
df_result['acertos'].sum()

8161

In [26]:
df_result.shape

(9500, 5)

### 4.0 Matriz de Confusão

In [27]:
mt.confusion_matrix(y_train, y_pred)

array([[ 369, 1136],
       [ 203, 7792]], dtype=int64)

### 4.1 - Acurácia

In [28]:
mt.accuracy_score(y_train, y_pred)

0.8590526315789474

In [14]:
y_train.shape

(9500,)

### 4.2 Precisão

In [31]:
# Precisão da label positiva 'Conceder'
mt.precision_score(y_train, y_pred, pos_label='Conceder' )

0.6451048951048951

### 4.3 Recall

In [32]:
# Recall da label positiva 'Conceder'
mt.recall_score(y_train, y_pred, pos_label='Conceder' )

0.24518272425249169